In [ ]:
!pip install transformers
!pip install datasets

In [2]:
from collections import defaultdict, Counter
import json

from matplotlib import pyplot as plt
import numpy as np
import torch

def print_encoding(model_inputs, indent=4):
    indent_str = " " * indent
    print("{")
    for k, v in model_inputs.items():
        print(indent_str + k + ":")
        print(indent_str + indent_str + str(v))
    print("}")

## Часть 0: Стандартные практики Hugging Face Transformers

Посмотрим на библиотеку Transformers на примере задачи sentiment analysis. 

Сперва найдем модель на [сайте](https://huggingface.co/models). Все желающие могут загружать туда свои обученные модели; наша модель описана в [этой статье](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3489963)).

Потом нам нужно инициализировать два объекта: токенизатор и модель.

![full_nlp_pipeline.png](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Инициализируем токенизатор: возьмем предобученный и положим в пустой класс-контейнер
tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")
# То же с моделью. Модель берем для классификации последовательностей (задача many-to-one)
model = AutoModelForSequenceClassification.from_pretrained("siebert/sentiment-roberta-large-english")

In [ ]:
# inference
inputs = "I'm excited to learn about Hugging Face Transformers!"
tokenized_inputs = tokenizer(inputs, return_tensors="pt") # возвращает id токенов словаря в тензорах, умеет в tf тоже
outputs = model(**tokenized_inputs) # распаковка словаря

labels = ['NEGATIVE', 'POSITIVE']
prediction = torch.argmax(outputs.logits)


print("Input:")
print(inputs)
print()
print("Tokenized Inputs:")
print_encoding(tokenized_inputs)
print()
print("Model Outputs:")
print(outputs)
print()
print(f"The prediction is {labels[prediction]}")

Attention mask показывает все слова, которые мы хотим учесть при расчете attention (мы не хотим включать в attention пады)

### 0.1 Токенизаторы

Обычно предобученные модели имеют при себе и соответствующие токенизаторы, которые нужны для предобработки инпутов. Токенизаторы принимают сырые строки или списки строк и возвращают словари, которые содержат инпуты модели. 

Можно подключать токенизаторы либо так, как выше (с помощью пустого класса AutoTokenizer), либо как в коде ниже: с помощью специфического для модели класса (тут у нас DistilBERT). У многих моделей есть два токенизатора: обычный (питоний) и быстрый (на расте). Правда, быстрые написали еще не для всех моделей.

In [ ]:
from transformers import DistilBertTokenizer, DistilBertTokenizerFast, AutoTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased")      # питоний
print(tokenizer)
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-cased")  # Rust
print(tokenizer)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased") # по умолчанию берет Rust, если тот доступен
print(tokenizer)

In [ ]:
# Так мы вызываем токенизатор
input_str = "Hugging Face Transformers is great!"
tokenized_inputs = tokenizer(input_str)


print("Vanilla Tokenization")
print_encoding(tokenized_inputs)
print()

# два способа посмотреть (как в pandas):
print(tokenized_inputs.input_ids)
print(tokenized_inputs["input_ids"])

BPE-токенизация в действии

In [ ]:
cls = [tokenizer.cls_token_id] # в нашем случае 101
sep = [tokenizer.sep_token_id] # 102

# Токенизация работает в несколько шагов:
input_tokens = tokenizer.tokenize(input_str) # собственно токенизировали
input_ids = tokenizer.convert_tokens_to_ids(input_tokens) # превратили в айдишники
input_ids_special_tokens = cls + input_ids + sep # добавили спецсимволы

decoded_str = tokenizer.decode(input_ids_special_tokens) # обратная операция

print("start:                ", input_str)
print("tokenize:             ", input_tokens)
print("convert_tokens_to_ids:", input_ids)
print("add special tokens:   ", input_ids_special_tokens)
print("--------")
print("decode:               ", decoded_str)

# Внимание: эти шаги не включают создание спецсимволов или аттеншн маски

In [ ]:
# У быстрых токенизаторов свой метод:
inputs = tokenizer._tokenizer.encode(input_str)

print(input_str)
print("-"*5)
print(f"Number of tokens: {len(inputs)}")
print(f"Ids: {inputs.ids}")
print(f"Tokens: {inputs.tokens}")
print(f"Special tokens mask: {inputs.special_tokens_mask}")
print()
print("char_to_word gives the wordpiece of a character in the input")
char_idx = 8
print(f"For example, the {char_idx + 1}th character of the string is '{input_str[char_idx]}',"+\
      f" and it's part of wordpiece {inputs.char_to_token(char_idx)}, '{inputs.tokens[inputs.char_to_token(char_idx)]}'")

In [ ]:
# Другие фишки:
# Токенизатор может возвращать торчевые тензоры (мы это уже видели выше)
model_inputs = tokenizer("Hugging Face Transformers is great!", return_tensors="pt")
print("PyTorch Tensors:")
print_encoding(model_inputs)

In [ ]:
# Можно передавать список строк и падить их, а заодно обрезать
model_inputs = tokenizer(["Hugging Face Transformers is great!",
                         "The quick brown fox jumps over the lazy dog." +\
                         "Then the dog got up and ran away because she didn't like foxes.",
                         ],
                         return_tensors="pt",
                         padding=True,
                         truncation=True)
print(f"Pad token: {tokenizer.pad_token} | Pad token id: {tokenizer.pad_token_id}")
print("Padding:")
print_encoding(model_inputs)

In [12]:
# Можно сразу весь батч отдекодить:
print("Batch Decode:")
print(tokenizer.batch_decode(model_inputs.input_ids))
print()
print("Batch Decode: (no special characters)")
print(tokenizer.batch_decode(model_inputs.input_ids, skip_special_tokens=True))

Batch Decode:
['[CLS] Hugging Face Transformers is great! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]', "[CLS] The quick brown fox jumps over the lazy dog. Then the dog got up and ran away because she didn't like foxes. [SEP]"]

Batch Decode: (no special characters)
['Hugging Face Transformers is great!', "The quick brown fox jumps over the lazy dog. Then the dog got up and ran away because she didn't like foxes."]


Больше документации тут:
[Hugging Face Transformers Docs](https://huggingface.co/docs/transformers/main_classes/tokenizer) и [Hugging Face Tokenizers Library](https://huggingface.co/docs/tokenizers/python/latest/quicktour.html) (про быстрые токенайзеры). Можно даже собственные токенизаторы учить с помощью библиотеки Tokenizers!

### 0.2 Модели


Инициализация моделей очень похожа на инициализацию токенизаторов. Можно либо использовать специальный класс модели, либо использовать AutoModel, в который впихнуть свою модель. Часто последний метод проще. 

Большинство предобученных трансформеров имеют похожую архитектуру, плюс в hugginface есть модельки с дополнительными весами (решающий линейный слой сверху), их еще часто называют головами (heads): они заточены решать какие-то конкретные downstream tasks. Hugging Face умеет автоматически выставлять ту архитектуру, которую вам нужно, когда вы определяете класс модели. Например, мы собираемся заняться sentiment analysis, значит, используем `DistilBertForSequenceClassification`. Если бы мы собирались продолжать дообучать берт на задаче masked language modelling, мы бы использовали  `DistilBertForMaskedLM`, а если мы просто хотели заполучить эмбеддинги из берта для какого-то кастомного таска, можно было бы использовать просто `DistilBertModel`.

Примерно так это выглядит в схемах: 
![model_illustration.png](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head.svg)


Пара примеров задач.
```
*
*ForMaskedLM
*ForSequenceClassification
*ForTokenClassification
*ForQuestionAnswering
*ForMultipleChoice
...
```
`*` может быть `AutoModel` или какая-то конкретная модель (например, `DistilBert`)


Всего у нас три типа моделей-трансформеров:
* Энкодеры (BERT)
* Декодеры (GPT)
* Энкодер+Декодер (BART или T5)

The task-specific classes you have available depend on what type of model you're dealing with.

Полный список [тут](https://huggingface.co/docs/transformers/model_doc/auto). Заметьте, что не все модели совместимы со всеми архитектурами, потому что чисто энкодеры решают одни задачи, а чисто декодеры - другие, например. Так, DistilBERT несовместим с архитектурой Seq2Seq, потому что у него нет декодера и masked attention. 


In [ ]:
from transformers import AutoModelForSequenceClassification, DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2) # количество классов в целевой переменной
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)

Предупреждение - абсолютно нормальное, просто говорит нам о том, что веса для sequence classification (верхний навешенный на нашего берта слой) еще не обучены. 

Передавать инпуты в модель супер просто. Тут есть два варианта, как это сделать:

In [ ]:
model_inputs = tokenizer(input_str, return_tensors="pt") # наш токенизатор возвращает айдишники + attention_mask

# Вариант 1 - передаем аргументы явно по ключу
model_outputs = model(input_ids=model_inputs.input_ids, attention_mask=model_inputs.attention_mask)

# Option 2 - ключи токенизатора совпадают с тем, чего ждет модель, поэтому можем просто распаковать словарь

# f({k1: v1, k2: v2}) = f(k1=v1, k2=v2)

model_outputs = model(**model_inputs)

print(model_inputs)
print()
print(model_outputs)
print()
print(f"Distribution over labels: {torch.softmax(model_outputs.logits, dim=1)}")

Если вы заметили, в задаче бинарной классификации мы вообще-то могли бы иметь только один аутпут и установить для него порог (типа, выхлоп от 0 до 1 и все, что меньше 0.5 - класс 0). Но так устроены модели huggingface, они по-своему высчитывают loss. 

Loss в модель можно передать свой, вычислить его отдельно как `loss_func` и вызвать `loss.backward`. Можно использовать любые оптимизаторы и шедулеры. 

In [ ]:
# Можно так вычислить лосс
label = torch.tensor([1])
loss = torch.nn.functional.cross_entropy(model_outputs.logits, label)
print(loss)
loss.backward()

# И глянуть параметры
list(model.named_parameters())[0]

Hugging Face модель может лосс непосредственно в аутпуте возвращать:

In [ ]:
# В таком случае мы должны передать метку класса:
model_inputs = tokenizer(input_str, return_tensors="pt")

labels = ['NEGATIVE', 'POSITIVE']
model_inputs['labels'] = torch.tensor([1])

model_outputs = model(**model_inputs)


print(model_outputs)
print()
print(f"Model predictions: {labels[model_outputs.logits.argmax()]}")

Можно также доставать из модели скрытые состояния и аттеншны - это полезно для анализа и просто разобрать все на досточки и посмотреть, что там внутри. (Например, [What does BERT look at?](https://arxiv.org/abs/1906.04341)).

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("distilbert-base-cased", output_attentions=True, output_hidden_states=True) # надо выставить флажки
model.eval()

model_inputs = tokenizer(input_str, return_tensors="pt")
with torch.no_grad():
    model_output = model(**model_inputs)


print("Hidden state size (per layer):  ", model_output.hidden_states[0].shape)
print("Attention head size (per layer):", model_output.attentions[0].shape)     # (layer, batch, query_word_idx, key_word_idxs)
                                                                               # y-axis is query, x-axis is key
print(model_output)    

Можно их поотрисовывать

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(model_inputs.input_ids[0])
print(tokens)


n_layers = len(model_output.attentions)
n_heads = len(model_output.attentions[0][0])
fig, axes = plt.subplots(6, 12)
fig.set_size_inches(18.5*2, 10.5*2)
for layer in range(n_layers):
    for i in range(n_heads):
        axes[layer, i].imshow(model_output.attentions[layer][0, i])
        axes[layer][i].set_xticks(list(range(9)))
        axes[layer][i].set_xticklabels(labels=tokens, rotation="vertical")
        axes[layer][i].set_yticks(list(range(9)))
        axes[layer][i].set_yticklabels(labels=tokens)

        if layer == 5:
            axes[layer, i].set(xlabel=f"head={i}")
        if i == 0:
            axes[layer, i].set(ylabel=f"layer={layer}")
            
plt.subplots_adjust(wspace=0.3)
plt.show()

## Часть 1: Finetuning

Скорее всего, нам захочется модель подучить. 

### 2.1 Загрузка датасета

В дополнение к моделям, [huggingface](https://huggingface.co/datasets) имеет и датасеты. 

In [ ]:
from datasets import load_dataset, DatasetDict

imdb_dataset = load_dataset("imdb")


# Возьмем только первые 50 токенов для скорости
def truncate(example):
    return {
        'text': " ".join(example['text'].split()[:50]),
        'label': example['label']
    }

# Возьмем 128 рандомных примеров для трейна и 32 для валидейшна
small_imdb_dataset = DatasetDict(
    train=imdb_dataset['train'].shuffle(seed=1111).select(range(128)).map(truncate),
    val=imdb_dataset['train'].shuffle(seed=1111).select(range(128, 160)).map(truncate),
)

In [ ]:
small_imdb_dataset

In [ ]:
small_imdb_dataset['train'][:10]

In [23]:
# Подготовим датасет - это токенизирует его и делит по батчам на 16 сэмплов
small_tokenized_dataset = small_imdb_dataset.map(
    lambda example: tokenizer(example['text'], padding=True, truncation=True),
    batched=True,
    batch_size=16
)

small_tokenized_dataset = small_tokenized_dataset.remove_columns(["text"])
small_tokenized_dataset = small_tokenized_dataset.rename_column("label", "labels")
small_tokenized_dataset.set_format("torch")

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [ ]:
small_tokenized_dataset['train'][0:2]

In [25]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_tokenized_dataset['train'], batch_size=16)
eval_dataloader = DataLoader(small_tokenized_dataset['val'], batch_size=16)

### 2.2 Обучение

Чтобы учить свои модельки, вы можете просто использовать стандартный трейнлуп ванильного торча. Модели Hugging Face - это тоже класс `torch.nn.Module`, так что бэкпроп работает ровно так же, и можно использовать те же оптимизаторы, что и обычно. Hugging Face также включает собственные оптимизаторы и шедулеры, которые использовались при обучении трансформеров, так что их тоже можно юзать. 

Для оптимизации будем использовать AdamW Optimizer - это Адам с плюшкой в виде weight decay. Также подключим линейный шедулер, который будет уменьшать lr понемножку после каждого шага трейнлупа. 

Есть и другие оптимизаторы и шедулеры, эти - дефолтные. Можно посмотреть на эти: [Hugging Face offers](https://huggingface.co/docs/transformers/main_classes/optimizer_schedules#schedules), или стандартные тут: [Pytorch](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate) (например, [ReduceLROnPlateau](https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ReduceLROnPlateau.html), который начинает снижать lr только тогда, когда val loss перестает падать), или даже собственные написать.

In [26]:
!mkdir checkpoints # создадим папку, куда торч будет сбрасывать чекпойнты - а то вдруг у вас отрубят свет.....

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm


model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)

num_epochs = 3
num_training_steps = 3 * len(train_dataloader)
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

best_val_loss = float("inf")
progress_bar = tqdm(range(num_training_steps))
for epoch in range(num_epochs):
    # training
    model.train()
    for batch_i, batch in enumerate(train_dataloader):
        
        # batch = ([text1, text2], [0, 1])

        output = model(**batch) # та самая распаковка инпутов и аттеншна
        
        optimizer.zero_grad()
        output.loss.backward()
        optimizer.step()
        lr_scheduler.step()
        progress_bar.update(1)
    
    # validation
    model.eval()
    for batch_i, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            output = model(**batch)
        loss += output.loss
    
    avg_val_loss = loss / len(eval_dataloader)
    print(f"Validation loss: {avg_val_loss}")
    if avg_val_loss < best_val_loss:
        print("Saving checkpoint!")
        best_val_loss = avg_val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': best_val_loss,
            },
            f"checkpoints/epoch_{epoch}.pt"
        )  

Еще у Hugging Face есть свой собственный класс для обучения, похожий на штуки, как в Lightning (если вы его еще не смотрели, посмотрите). Можно не писать свои трейнлупы за пупы!

`TrainingArguments` определяет различные параметры обучения, типа как часто оценивать и бэкапить модельки, куда их бэкапить и т.п. Настраивать можно много всего, можно посмотреть [тут](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments). Некоторые подконтрольные вещи включают:
* learning rate, weight decay, gradient clipping, 
* checkpointing, logging, and evaluation frequency
* where you log to (default is tensorboard, but if you use WandB or MLFlow they have integrations)

Класс `Trainer` собственно обучает модель. Можно в него передать `TrainingArguments`, модель, датасеты, токенизатор, оптимайзер и даже чекпойнты модели, если у вас все-таки отрубили свет... Функция `compute_metrics` вызывается в конце evaluation/validation, чтобы посчитать метрики. 

In [ ]:
from transformers import TrainingArguments, Trainer

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)

arguments = TrainingArguments(
    output_dir="sample_hf_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch", # валидация в конце каждой эпохи
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    seed=224
)


def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return {"accuracy": np.mean(predictions == labels)}


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'], # для финальной оценки меняем на test
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Некоторые практические советы для файнтьюнинга:

**Хорошие дефолтные параметры.** Гиперпараметры зависят от задачи и датасета. Хорошо бы погридсерчить на лучшие, но есть практикой установленные хорошие стартовые значения. 
* Эпохи: {2, 3, 4} (чем больше данных, тем меньше надо эпох)
* Размер батча (чем больше, тем быстрее и стабильнее)
* Оптимизатор: AdamW
* AdamW learning rate: {2e-5, 5e-5}
* Learning rate scheduler: linear warm up for first {0, 100, 500} steps of training
* weight_decay (l2 regularization): {0, 0.01, 0.1}

Нужно следить за validation loss, чтобы понять, какие параметры хорошие. 

## Приложение: Пайплайны

Для некоторых стандартных NLP задач, типа sentiment classification или QA, есть уже полностью готовые модели, которые доступны в интерфейсе [Pipeline](https://huggingface.co/docs/transformers/v4.16.2/en/main_classes/pipelines#transformers.pipeline) Hugging Face. 

Они, может быть, менее интересны, но знать про них стоит. 

Вот примерчик:

In [30]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")

Пайплайн просто гоняется по строке инпута

In [ ]:
sentiment_analysis("Hugging Face Transformers is really cool!")

Или списке строк

In [ ]:
sentiment_analysis(["I didn't know if I would like Hákarl, but it turned out pretty good.",
                    "I didn't know if I would like Hákarl, and it was just as bad as I'd heard."])